In [1]:
import pandas as pd 
import numpy as np 
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import datetime
import pyodbc
import xlwings as xw
import os
import time

US_BUS_DAY = CustomBusinessDay()#(calendar=USFederalHolidayCalendar())
today = datetime.date.today()
result = today - 0 * US_BUS_DAY
next_bus_day = result.strftime('%m-%d-%Y')
print(next_bus_day)


cnxn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=e360-db01;'
    r'DATABASE=Voltage;'
    r'Trusted_Connection=yes;'
)


def compute_etl(losses, confidence_level):
    var_threshold = np.percentile(losses, 100 - confidence_level)
    tail_losses = [loss for loss in losses if loss < var_threshold]
    print(f'These are the tail losses for {losses.name}: {tail_losses}')
    if len(tail_losses) == 0:
        return None
    else:
        return np.mean(tail_losses)
    
    
companies = ['E360 POWER FUND LP', 'OCTOBER FUTURES, LLC', 'MOROSS']

01-08-2024


### By Company

In [2]:
# Loop over the companies list
for company in companies:
    print(company)
    hvar_vectors_q = f"""
    --- Get HVaR Vectors and Portfolio
    declare @as_of_date date = '{next_bus_day}';
    declare @company varchar(255) = '{company}';
    WITH PivotCTE AS (
      SELECT
        historical_date,
        RIGHT(CAST(MONTH(historical_date) AS VARCHAR), 2) + 
        RIGHT('0' + CAST(DAY(historical_date) AS VARCHAR), 2) + 
        CAST(YEAR(historical_date) AS VARCHAR) AS formatted_date,
        [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
        [IPJO], [ITMT], [IERN]
      FROM (
        SELECT [historical_date], [bucket_name], [hist_pnl]
        FROM hvar_vectors_percent
        WHERE as_of_date = @as_of_date
        AND company = @company
      ) AS PivotData
      PIVOT (
        SUM([hist_pnl])
        FOR [bucket_name] IN ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN])
      ) AS PivotTable
    )
    SELECT formatted_date As ' ',
           COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
           COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
           COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
           COALESCE([IERN], 0) IERN,
           COALESCE([IHHS], 0) + COALESCE([IWAH], 0) + COALESCE([ITTF], 0) +
           COALESCE([IBRN], 0) + COALESCE([IWTI], 0) + COALESCE([IPJM], 0) +
           COALESCE([IP6O], 0) + COALESCE([IPJO], 0) + COALESCE([ITMT], 0) +
           COALESCE([IERN], 0) AS Portfolio
    FROM PivotCTE
    ORDER BY historical_date DESC;
    """


    hvar_percentile_buckets_q = f"""
    --- Get HVaR Percentile across buckets
    declare @as_of_date date = '{next_bus_day}';
    declare @company varchar(255) = '{company}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
               [IPJO], [ITMT], [IERN]
        FROM (
            SELECT as_of_date, bucket_name, percentile_99
            FROM hvar_percentiles_percent
            WHERE as_of_date = @as_of_date
            AND company = @company
            ) AS PivotData
            PIVOT (
                sum(percentile_99)
                FOR bucket_name in ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
               [IPJO], [ITMT], [IERN])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
           COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
           COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
           COALESCE([IERN], 0) IERN
    FROM PivotCTE;
    """


    hvar_etl_buckets_q = f"""
    --- Get ETL across buckets
    declare @as_of_date date = '{next_bus_day}';
    declare @company varchar(255) = '{company}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
               [IPJO], [ITMT], [IERN]
        FROM (
            SELECT as_of_date, bucket_name, etl_cvar
            FROM hvar_etl_data
            WHERE as_of_date = @as_of_date
            AND company = @company
            ) AS PivotData
            PIVOT (
                sum(etl_cvar)
                FOR bucket_name in ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
               [IPJO], [ITMT], [IERN])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
           COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
           COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
           COALESCE([IERN], 0) IERN
    FROM PivotCTE;
    """


    portfolio_hvar_q = f"""
    --- Get Portfolio HVaR 
    declare @as_of_date date = '{next_bus_day}';
    declare @company varchar(255) = '{company}';
    select percentile_99 from hvar_percentiles_percent
    where as_of_date = @as_of_date
    and bucket_name = 'Total Portfolio'
    and company = @company
    """
    
    etl_real_var_q = f"""
    select historical_date, sum(hist_pnl) hist_pnl from hvar_vectors_percent
        where company = '{company}'
        and as_of_date = '{next_bus_day}'
        group by historical_date
    """
    
    hvar_vectors = pd.read_sql(hvar_vectors_q, cnxn)
    display(hvar_vectors)
    
    etl_real_var = pd.read_sql(etl_real_var_q, cnxn)
    display(etl_real_var)
    etl_real_var.dropna(inplace = True)


    portfolio_hvar = pd.read_sql(portfolio_hvar_q, cnxn)
    port_var = portfolio_hvar.percentile_99[0]
    port_etl = compute_etl(etl_real_var['hist_pnl'], 99)

    hvar_percentile_buckets = pd.read_sql(hvar_percentile_buckets_q, cnxn)
    percent_buckets = hvar_percentile_buckets.iloc[:,1:]
    percent_buckets['Portfolio VaR'] = port_var
    percent_buckets['Portfolio ETL'] = port_etl
    display(percent_buckets)


    hvar_etl_buckets = pd.read_sql(hvar_etl_buckets_q, cnxn)
    etl_buckets = hvar_etl_buckets.iloc[:,1:]
    display(etl_buckets)

    # Create a new Excel workbook
    wb = xw.Book()

    # Add the dataframes to the sheets in the workbook
    wb.sheets.add(name='Daily_Hist_Var', after=wb.sheets[0]).range('A1').options(index=False).value = hvar_vectors
    wb.sheets.add(name='VaR', after=wb.sheets[1]).range('A1').options(index=False).value = percent_buckets
    wb.sheets['VaR'].range('A7').options(index=False).value = etl_buckets

    # Save the file with company name and date in the filename
    if company == companies[2]:
        file_name = f"Moross_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
        file_path = os.path.join(r"Y:\work_ajo\VaR\Moross", file_name)
    elif company == companies[1]:
        file_name = f"OctFut_Full_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
        file_path = os.path.join(r"Y:\work_ajo\VaR\OctFut_Full", file_name)
    elif company == companies[0]:
        file_name = f"e360Fund_Full_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
        file_path = os.path.join(r"Y:\work_ajo\VaR\e360Fund_Full", file_name)
        
        
    print(file_path)
    wb.save(file_path)

    # Close the workbook and quit Excel
    wb.close()
    xw.App().quit()
    print('done')
    time.sleep(1)


E360 POWER FUND LP


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2882036068.py:115: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_vectors = pd.read_sql(hvar_vectors_q, cnxn)


,,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN,Portfolio
0,1042024,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00
1,1032024,-2.276965e+06,0.0,-127985.645752,0.0,0.0,1.149279e+06,-1.530834e+05,-57068.532465,0.0,4.501292e+05,-1.015694e+06
2,1022024,-3.319966e+06,0.0,86803.691276,0.0,0.0,3.610942e+05,1.437146e+04,35320.664766,0.0,6.979122e+04,-2.752585e+06
3,1012024,2.932993e+02,0.0,882.339680,0.0,0.0,-1.299772e+03,-8.229422e+03,0.000000,0.0,-1.819914e+03,-1.017347e+04
4,12292023,9.845026e+05,0.0,35929.315472,0.0,0.0,-5.899277e+05,2.252639e+04,-11185.153268,0.0,-1.643280e+05,2.775174e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
254,1132023,5.380922e+06,0.0,-334217.798329,0.0,0.0,-2.597466e+06,-2.642439e+05,-415227.337003,0.0,-7.122836e+05,1.057483e+06
255,1122023,-1.244185e+06,0.0,97644.871008,0.0,0.0,7.667517e+05,2.121597e+05,18855.009537,0.0,6.790363e+05,5.302628e+05
256,1112023,-1.570941e+06,0.0,256070.209392,0.0,0.0,-2.398147e+06,1.828390e+05,-412233.439022,0.0,-1.978821e+05,-4.140295e+06
257,1102023,8.810999e+06,0.0,149490.178403,0.0,0.0,-4.633639e+06,-6.882697e+05,-264500.900873,0.0,-1.526271e+06,1.847809e+06


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2882036068.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  etl_real_var = pd.read_sql(etl_real_var_q, cnxn)


,historical_date,hist_pnl
0,2023-02-08,3.523796e+06
1,2023-02-09,-5.216959e+04
2,2023-02-23,-5.232118e+06
3,2023-03-13,-4.931125e+06
4,2023-03-30,2.097129e+06
...,...,...
254,2023-11-07,1.865290e+06
255,2023-11-14,1.303437e+06
256,2023-12-20,1.735355e+06
257,2023-12-27,3.353144e+05


...

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2882036068.py:123: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  portfolio_hvar = pd.read_sql(portfolio_hvar_q, cnxn)
C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2882036068.py:127: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_percentile_buckets = pd.read_sql(hvar_percentile_buckets_q, cnxn)


,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN,Portfolio VaR,Portfolio ETL
0,-1.859123e+06,0.0,-67540.180681,0.0,0.0,-694664.165893,-33557.524823,-125792.556031,0.0,-231709.216876,-1.498375e+06,-1.701935e+06


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2882036068.py:134: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_etl_buckets = pd.read_sql(hvar_etl_buckets_q, cnxn)


,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN
0,-2124846.0,0.0,-89718.882812,0.0,0.0,-828190.875,-44447.636719,-145919.46875,0.0,-317224.3125


Y:\work_ajo\VaR\Moross\Moross_VaR_Output_01082024.xlsx
done


### JPJS Only

In [3]:
hvar_vectors_q = f"""
--- Get HVaR Vectors and Portfolio
declare @as_of_date date = '{next_bus_day}';
    WITH hvar_vectors_percent_cte as (
	select as_of_date, historical_date, bucket_name, 'Total Portfolio' as company, sum(hist_pnl) hist_pnl  from hvar_vectors_percent
		where as_of_date  = @as_of_date
		group by as_of_date, historical_date, bucket_name
	),	
	PivotCTE AS (
      SELECT
        historical_date,
        RIGHT(CAST(MONTH(historical_date) AS VARCHAR), 2) + 
        RIGHT('0' + CAST(DAY(historical_date) AS VARCHAR), 2) + 
        CAST(YEAR(historical_date) AS VARCHAR) AS formatted_date,
        [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
        [IPJO], [ITMT], [IERN]
      FROM (
        SELECT [historical_date], [bucket_name], [hist_pnl]
        FROM hvar_vectors_percent_cte
        WHERE as_of_date = @as_of_date
      ) AS PivotData
      PIVOT (
        SUM([hist_pnl])
        FOR [bucket_name] IN ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN])
      ) AS PivotTable
    )
    SELECT formatted_date As ' ',
           COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
           COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
           COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
           COALESCE([IERN], 0) IERN,
           COALESCE([IHHS], 0) + COALESCE([IWAH], 0) + COALESCE([ITTF], 0) +
           COALESCE([IBRN], 0) + COALESCE([IWTI], 0) + COALESCE([IPJM], 0) +
           COALESCE([IP6O], 0) + COALESCE([IPJO], 0) + COALESCE([ITMT], 0) +
           COALESCE([IERN], 0) AS Portfolio
    FROM PivotCTE
    ORDER BY historical_date DESC;
"""


hvar_percentile_buckets_q = f"""
--- Get HVaR Percentile across buckets
declare @as_of_date date = '{next_bus_day}';
declare @company varchar(255) = 'Portfolio Total';
WITH PivotCTE AS (
    SELECT as_of_date,
           [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN]
    FROM (
        SELECT as_of_date, bucket_name, percentile_99
        FROM hvar_percentiles_percent
        WHERE as_of_date = @as_of_date
        AND company = @company
        ) AS PivotData
        PIVOT (
            sum(percentile_99)
            FOR bucket_name in ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN])
      ) AS PivotTable
 )
SELECT as_of_date, COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
       COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
       COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
       COALESCE([IERN], 0) IERN
FROM PivotCTE;
"""


hvar_etl_buckets_q = f"""
--- Get ETL across buckets
declare @as_of_date date = '{next_bus_day}';
declare @company varchar(255) = 'Total Portfolio';
WITH PivotCTE AS (
    SELECT as_of_date,
           [IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN]
    FROM (
        SELECT as_of_date, bucket_name, etl_cvar
        FROM hvar_etl_data
        WHERE as_of_date = @as_of_date
        AND company = @company
        ) AS PivotData
        PIVOT (
            sum(etl_cvar)
            FOR bucket_name in ([IHHS], [IWAH], [ITTF], [IBRN], [IWTI], [IPJM], [IP6O],
           [IPJO], [ITMT], [IERN])
      ) AS PivotTable
 )
SELECT as_of_date, COALESCE([IHHS], 0) IHHS, COALESCE([IWAH], 0) IWAH, COALESCE([ITTF], 0) ITTF,
       COALESCE([IBRN], 0) IBRN, COALESCE([IWTI], 0) IWTI, COALESCE([IPJM], 0) IPJM,
       COALESCE([IP6O], 0) IP6O, COALESCE([IPJO], 0) IPJO, COALESCE([ITMT], 0) ITMT,
       COALESCE([IERN], 0) IERN
FROM PivotCTE;
"""


portfolio_hvar_q = f"""
--- Get Portfolio HVaR 
declare @as_of_date date = '{next_bus_day}';
declare @company varchar(255) = 'Portfolio Total';
select percentile_99 from hvar_percentiles_percent
where as_of_date = @as_of_date
and bucket_name = 'Total Portfolio'
and company = @company
"""

etl_real_var_q = f"""
select historical_date, sum(hist_pnl) hist_pnl from hvar_vectors_percent
    where as_of_date = '{next_bus_day}'
    group by historical_date
"""

In [4]:
hvar_vectors = pd.read_sql(hvar_vectors_q, cnxn)
display(hvar_vectors)

etl_real_var = pd.read_sql(etl_real_var_q, cnxn)
display(etl_real_var)
etl_real_var.dropna(inplace = True)


portfolio_hvar = pd.read_sql(portfolio_hvar_q, cnxn)
port_var = portfolio_hvar.percentile_99[0]
port_etl = compute_etl(etl_real_var['hist_pnl'], 99)

hvar_percentile_buckets = pd.read_sql(hvar_percentile_buckets_q, cnxn)
percent_buckets = hvar_percentile_buckets.iloc[:,1:]
percent_buckets['Portfolio VaR'] = port_var
percent_buckets['Portfolio ETL'] = port_etl
display(percent_buckets)


hvar_etl_buckets = pd.read_sql(hvar_etl_buckets_q, cnxn)
etl_buckets = hvar_etl_buckets.iloc[:,1:]
display(etl_buckets)

# Create a new Excel workbook
wb = xw.Book()

# Add the dataframes to the sheets in the workbook
wb.sheets.add(name='Daily_Hist_Var', after=wb.sheets[0]).range('A1').options(index=False).value = hvar_vectors
wb.sheets.add(name='VaR', after=wb.sheets[1]).range('A1').options(index=False).value = percent_buckets
wb.sheets['VaR'].range('A7').options(index=False).value = etl_buckets

# Save the file with company name and date in the filename
file_name = f"e360Power_JPJS_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
file_path = os.path.join(r"Y:\work_ajo\VaR\e360Power_JPJS", file_name)

print(file_path)
wb.save(file_path)

# Close the workbook and quit Excel
wb.close()
xw.App().quit()
print('done')
time.sleep(1)

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\318495520.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_vectors = pd.read_sql(hvar_vectors_q, cnxn)


,,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN,Portfolio
0,1042024,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00
1,1032024,-5.020841e+06,0.0,-161694.855938,0.0,0.0,2.394595e+06,-3.270315e+05,-9.915733e+04,0.0,7.602022e+04,-3.138110e+06
2,1022024,-7.362118e+06,0.0,111043.717085,0.0,0.0,8.199382e+05,1.437032e+04,6.591792e+04,0.0,1.628907e+05,-6.187957e+06
3,1012024,6.300927e+02,0.0,1063.059855,0.0,0.0,-1.888324e+03,-1.651401e+04,0.000000e+00,0.0,-3.014587e+03,-1.972376e+04
4,12292023,2.171308e+06,0.0,47484.921364,0.0,0.0,-1.326515e+06,7.186849e+04,-6.184513e+04,0.0,-2.531940e+05,6.491073e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
254,1132023,1.209407e+07,0.0,-397826.377641,0.0,0.0,-5.772502e+06,-4.392408e+05,-1.039392e+06,0.0,-8.149111e+05,3.630193e+06
255,1122023,-2.788002e+06,0.0,115800.634185,0.0,0.0,1.697690e+06,5.333678e+05,9.306042e+04,0.0,8.662797e+05,5.181964e+05
256,1112023,-3.546464e+06,0.0,315925.172036,0.0,0.0,-5.702060e+06,2.474050e+05,-9.979437e+05,0.0,-3.222479e+05,-1.000539e+07
257,1102023,1.973821e+07,0.0,185983.671356,0.0,0.0,-1.069978e+07,-9.452101e+05,-6.973805e+05,0.0,-2.943016e+06,4.638811e+06


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\318495520.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  etl_real_var = pd.read_sql(etl_real_var_q, cnxn)


,historical_date,hist_pnl
0,2023-01-17,-9.917014e+05
1,2023-04-04,1.053007e+07
2,2023-12-26,1.837881e+06
3,2023-07-13,4.532117e+06
4,2023-02-20,-1.325634e+04
...,...,...
254,2023-07-24,-1.423303e+06
255,2023-07-07,5.822628e+06
256,2023-11-01,2.010344e+06
257,2023-10-18,3.103903e+06


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\318495520.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  portfolio_hvar = pd.read_sql(portfolio_hvar_q, cnxn)
C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\318495520.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_percentile_buckets = pd.read_sql(hvar_percentile_buckets_q, cnxn)


These are the tail losses for hist_pnl: [-20178825.947722804, -16993775.860686883, -17555462.33884205]


,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN,Portfolio VaR,Portfolio ETL
0,-2.051089e+07,0.0,-357728.71307,0.0,0.0,-7.266528e+06,-403226.01576,-1.362886e+06,0.0,-2.257521e+06,-1.658536e+07,-1.824269e+07


C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\318495520.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hvar_etl_buckets = pd.read_sql(hvar_etl_buckets_q, cnxn)


,IHHS,IWAH,ITTF,IBRN,IWTI,IPJM,IP6O,IPJO,ITMT,IERN
0,-23249728.0,0.0,-454384.53125,0.0,0.0,-8631409.0,-610587.75,-1562041.5,0.0,-2498813.25


Y:\work_ajo\VaR\e360Power_JPJS\e360Power_JPJS_VaR_Output_01082024.xlsx
done


### NG Long 1 Lot (NG_1Lot)

In [5]:
gas_lots_long_vectors = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
      SELECT
        historical_date,
        RIGHT(CAST(MONTH(historical_date) AS VARCHAR), 2) + 
        RIGHT('0' + CAST(DAY(historical_date) AS VARCHAR), 2) + 
        CAST(YEAR(historical_date) AS VARCHAR) AS formatted_date,
        [GAS LOT LONG]
      FROM (
        SELECT [historical_date], [bucket_name], [hist_pnl]
        FROM hvar_vectors_percent_gas_lots
        WHERE as_of_date = @as_of_date
      ) AS PivotData
      PIVOT (
        SUM([hist_pnl])
        FOR [bucket_name] IN ([GAS LOT LONG])
      ) AS PivotTable
    )
    SELECT formatted_date As ' ',
           COALESCE([GAS LOT LONG], 0) IHHS,
           COALESCE([GAS LOT LONG], 0) AS Portfolio
    FROM PivotCTE
    ORDER BY historical_date DESC;
"""

gas_lots_long_percentiles = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [GAS LOT LONG]
        FROM (
            SELECT as_of_date, bucket_name, percentile_99
            FROM hvar_percentiles_percent_gas_lots
            WHERE as_of_date = @as_of_date
            ) AS PivotData
            PIVOT (
                sum(percentile_99)
                FOR bucket_name in ([GAS LOT LONG])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([GAS LOT LONG], 0) IHHS
    FROM PivotCTE;
"""

gas_lots_long_etl = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [GAS LOT LONG]
        FROM (
            SELECT as_of_date, bucket_name, etl_cvar
            FROM hvar_etl_data
            WHERE as_of_date = @as_of_date
            ) AS PivotData
            PIVOT (
                sum(etl_cvar)
                FOR bucket_name in ([GAS LOT LONG])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([GAS LOT LONG], 0) IHHS
    FROM PivotCTE;
"""

In [6]:
gas_lots_long_ETL = pd.read_sql(gas_lots_long_etl, cnxn)
gas_lots_long_ETL

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\533004639.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_long_ETL = pd.read_sql(gas_lots_long_etl, cnxn)


,as_of_date,IHHS
0,2024-01-08,-0.295734


In [7]:
etl_buckets = gas_lots_long_ETL.drop('as_of_date', axis = 1)
etl_buckets

,IHHS
0,-0.295734


In [8]:
gas_lots_long_PERCENTILES = pd.read_sql(gas_lots_long_percentiles, cnxn)
gas_lots_long_PERCENTILES

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\917553523.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_long_PERCENTILES = pd.read_sql(gas_lots_long_percentiles, cnxn)


,as_of_date,IHHS
0,2024-01-08,-0.220825


In [9]:
percent_buckets = gas_lots_long_PERCENTILES.drop('as_of_date', axis = 1)
percent_buckets['Portfolio VaR'] = gas_lots_long_PERCENTILES.IHHS[0].sum()
percent_buckets['Portfolio ETL'] = gas_lots_long_ETL.IHHS[0].sum()
percent_buckets

,IHHS,Portfolio VaR,Portfolio ETL
0,-0.220825,-0.220825,-0.295734


In [10]:
gas_lots_long_VECTORS = pd.read_sql(gas_lots_long_vectors, cnxn)
gas_lots_long_VECTORS

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\2334570413.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_long_VECTORS = pd.read_sql(gas_lots_long_vectors, cnxn)


,,IHHS,Portfolio
0,1042024,0.161774,0.161774
1,1032024,0.109852,0.109852
2,1022024,0.060594,0.060594
3,1012024,0.000000,0.000000
4,12292023,-0.047440,-0.047440
...,...,...,...
254,1132023,-0.210716,-0.210716
255,1122023,0.018443,0.018443
256,1112023,0.024807,0.024807
257,1102023,-0.195522,-0.195522


In [11]:
# Create a new Excel workbook
wb = xw.Book()

# Add the dataframes to the sheets in the workbook
wb.sheets.add(name='Daily_Hist_Var', after=wb.sheets[0]).range('A1').options(index=False).value = gas_lots_long_VECTORS
wb.sheets.add(name='VaR', after=wb.sheets[1]).range('A1').options(index=False).value = percent_buckets
wb.sheets['VaR'].range('A7').options(index=False).value = etl_buckets

# Save the file with company name and date in the filename
file_name = f"NG_1Lot_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
file_path = os.path.join(r"Y:\work_ajo\VaR\NG_1Lot", file_name)


print(file_path)
wb.save(file_path)

# Close the workbook and quit Excel
wb.close()
xw.App().quit()
print('done')
time.sleep(1)

Y:\work_ajo\VaR\NG_1Lot\NG_1Lot_VaR_Output_01082024.xlsx
done


### NG Short 1 Lot (NG-1Lot)

In [12]:
gas_lots_short_vectors = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
      SELECT
        historical_date,
        RIGHT(CAST(MONTH(historical_date) AS VARCHAR), 2) + 
        RIGHT('0' + CAST(DAY(historical_date) AS VARCHAR), 2) + 
        CAST(YEAR(historical_date) AS VARCHAR) AS formatted_date,
        [GAS LOT SHORT]
      FROM (
        SELECT [historical_date], [bucket_name], [hist_pnl]
        FROM hvar_vectors_percent_gas_lots
        WHERE as_of_date = @as_of_date
      ) AS PivotData
      PIVOT (
        SUM([hist_pnl])
        FOR [bucket_name] IN ([GAS LOT SHORT])
      ) AS PivotTable
    )
    SELECT formatted_date As ' ',
           COALESCE([GAS LOT SHORT], 0) IHHS,
           COALESCE([GAS LOT SHORT], 0) AS Portfolio
    FROM PivotCTE
    ORDER BY historical_date DESC;
"""

gas_lots_short_percentiles = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [GAS LOT SHORT]
        FROM (
            SELECT as_of_date, bucket_name, percentile_99
            FROM hvar_percentiles_percent_gas_lots
            WHERE as_of_date = @as_of_date
            ) AS PivotData
            PIVOT (
                sum(percentile_99)
                FOR bucket_name in ([GAS LOT SHORT])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([GAS LOT SHORT], 0) IHHS
    FROM PivotCTE;
"""

gas_lots_short_etl = f"""
declare @as_of_date date = '{next_bus_day}';
    WITH PivotCTE AS (
        SELECT as_of_date,
               [GAS LOT SHORT]
        FROM (
            SELECT as_of_date, bucket_name, etl_cvar
            FROM hvar_etl_data
            WHERE as_of_date = @as_of_date
            ) AS PivotData
            PIVOT (
                sum(etl_cvar)
                FOR bucket_name in ([GAS LOT SHORT])
          ) AS PivotTable
     )
    SELECT as_of_date, COALESCE([GAS LOT SHORT], 0) IHHS
    FROM PivotCTE;
"""

In [13]:
gas_lots_short_ETL = pd.read_sql(gas_lots_short_etl, cnxn)
gas_lots_short_ETL

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\952395434.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_short_ETL = pd.read_sql(gas_lots_short_etl, cnxn)


,as_of_date,IHHS
0,2024-01-08,-0.271378


In [14]:
etl_buckets = gas_lots_short_ETL.drop('as_of_date', axis = 1)
etl_buckets

,IHHS
0,-0.271378


In [15]:
gas_lots_short_PERCENTILES = pd.read_sql(gas_lots_short_percentiles, cnxn)
gas_lots_short_PERCENTILES

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\1295345708.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_short_PERCENTILES = pd.read_sql(gas_lots_short_percentiles, cnxn)


,as_of_date,IHHS
0,2024-01-08,-0.252787


In [16]:
percent_buckets = gas_lots_short_PERCENTILES.drop('as_of_date', axis = 1)
percent_buckets['Portfolio VaR'] = gas_lots_short_PERCENTILES.IHHS[0].sum()
percent_buckets['Portfolio ETL'] = gas_lots_short_ETL.IHHS[0].sum()
percent_buckets

,IHHS,Portfolio VaR,Portfolio ETL
0,-0.252787,-0.252787,-0.271378


In [17]:
gas_lots_short_VECTORS = pd.read_sql(gas_lots_short_vectors, cnxn)
gas_lots_short_VECTORS

C:\Users\kjones\AppData\Local\Temp\ipykernel_17272\356496321.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gas_lots_short_VECTORS = pd.read_sql(gas_lots_short_vectors, cnxn)


,,IHHS,Portfolio
0,1042024,-0.161774,-0.161774
1,1032024,-0.109852,-0.109852
2,1022024,-0.060594,-0.060594
3,1012024,0.000000,0.000000
4,12292023,0.047440,0.047440
...,...,...,...
254,1132023,0.210716,0.210716
255,1122023,-0.018443,-0.018443
256,1112023,-0.024807,-0.024807
257,1102023,0.195522,0.195522


In [18]:
# Create a new Excel workbook
wb = xw.Book()

# Add the dataframes to the sheets in the workbook
wb.sheets.add(name='Daily_Hist_Var', after=wb.sheets[0]).range('A1').options(index=False).value = gas_lots_short_VECTORS
wb.sheets.add(name='VaR', after=wb.sheets[1]).range('A1').options(index=False).value = percent_buckets
wb.sheets['VaR'].range('A7').options(index=False).value = etl_buckets

# Save the file with company name and date in the filename
file_name = f"NG-1Lot_VaR_Output_{next_bus_day.replace('-', '')}.xlsx"
file_path = os.path.join(r"Y:\work_ajo\VaR\NG-1Lot", file_name)


print(file_path)
wb.save(file_path)

# Close the workbook and quit Excel
wb.close()
xw.App().quit()
print('done')
time.sleep(1)

Y:\work_ajo\VaR\NG-1Lot\NG-1Lot_VaR_Output_01082024.xlsx
done
